##### Combine CSV Files

In [7]:
# import libraries
import pandas as pd
import os
import indra_cogex.sources.nih_reporter
import gilda
import nltk
import json

In [8]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/owensharpe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

##### Get Clinical Trials and Patents initially because there's only one csv each

In [9]:
# get patents and clinical trials
clinical_trials = pd.read_csv('/Users/owensharpe/Desktop/Analyzing NIH Database/data collection/nih_reporter_website_data/ClinicalStudies_1735707600.csv')
patents = pd.read_csv('/Users/owensharpe/Desktop/Analyzing NIH Database/data collection/nih_reporter_website_data/Patents_1735707600.csv')

In [10]:
# loop through the data folder and create list of dataframes of project, publication, and abstract data
project_list, publication_list, abstract_list = [], [], []

# create file path
path = '/Users/owensharpe/Desktop/Analyzing NIH Database/data collection/nih_reporter_website_data'

# create instance of the NIHReporterProcessor class
nih = indra_cogex.sources.nih_reporter

for file in os.listdir(path):
    if file.endswith(".zip"):
        if "PRJ_C" in file:  # check if project
            zip_path = os.path.join(path, file)
            temp_df = nih._read_first_df(zip_path)
            project_list.append(temp_df)
        elif "PUBLINK_C" in file:  # check if publications
            zip_path = os.path.join(path, file)
            temp_df = nih._read_first_df(zip_path)
            publication_list.append(temp_df)
        elif "PRJABS_C" in file:  # check if abstract
            zip_path = os.path.join(path, file)
            temp_df = nih._read_first_df(zip_path)
            abstract_list.append(temp_df)
    
# create large dataframes    
projects = pd.concat(project_list)
publications = pd.concat(publication_list)
abstracts = pd.concat(abstract_list)

In [11]:
projects

,APPLICATION_ID,ACTIVITY,ADMINISTERING_IC,APPLICATION_TYPE,ARRA_FUNDED,AWARD_NOTICE_DATE,BUDGET_START,BUDGET_END,CFDA_CODE,CORE_PROJECT_NUM,...,STUDY_SECTION,STUDY_SECTION_NAME,SUBPROJECT_ID,SUFFIX,SUPPORT_YEAR,DIRECT_COST_AMT,INDIRECT_COST_AMT,TOTAL_COST,TOTAL_COST_SUB_PROJECT,FOA_NUMBER
0,6465311,K22,CA,1.0,N,2007-04-03,2007-04-03,2008-03-31,398.0,K22CA095325,...,ZCA1,ZCA1-GRB-J(J1),NaN,NaN,1.0,147875.0,11830.0,159705.0,NaN,NaN
1,6626317,F32,AG,5.0,N,2007-04-12,2007-05-01,2008-04-30,866.0,F32AG020903,...,ZRG1,Special Emphasis Panel[ZRG1-F01(20)L],NaN,NaN,2.0,58036.0,NaN,58036.0,NaN,NaN
2,6634305,U19,DA,5.0,N,2007-06-26,2007-06-01,2010-05-31,279.0,U19DA013326,...,ZDA1,ZDA1-KXA-N(18),NaN,NaN,4.0,787545.0,87519.0,650551.0,NaN,NaN
3,6803975,R21,CA,5.0,N,2007-06-18,2007-06-18,2010-05-31,395.0,R21CA105705,...,ZRG1,Special Emphasis Panel[ZRG1-ET-1(03)],NaN,NaN,2.0,216048.0,115586.0,331634.0,NaN,NaN
4,6806737,U44,DK,4.0,N,2006-12-11,2006-12-15,2007-11-30,847.0,U44DK066724,...,ZRG1,Special Emphasis Panel[ZRG1-SSS-D(10)B],NaN,NaN,3.0,1587848.0,239456.0,1948453.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83843,11078400,R21,AG,7.0,N,2024-05-27,2024-05-01,2025-04-30,866.0,R21AG070714,...,EITA,Emerging Imaging Technologies and Applications...,NaN,NaN,3.0,54099.0,36246.0,90345.0,NaN,NaN
83844,11079423,U01,GH,6.0,N,2024-04-23,2023-09-01,2024-08-31,326.0,U01GH002290,...,ZGH1,ZGH1-HMS(02),NaN,M003,4.0,NaN,NaN,221500.0,NaN,NaN
83845,11081606,U01,GH,6.0,N,2024-04-26,2022-09-30,2024-09-29,326.0,U01GH002243,...,ZGH1,ZGH1-HMS(01),NaN,M008,5.0,NaN,NaN,72925.0,NaN,NaN
83846,11111898,DP2,DA,7.0,N,2024-07-05,2023-10-27,2026-03-31,279.0,DP2DA056172,...,ZDA1,ZDA1-YXF-U(06)S,NaN,NaN,2.0,750000.0,393750.0,1143750.0,NaN,NaN


##### Use Gilda Library to Annotate Abstract Text Data

In [12]:
abstracts = abstracts[~pd.isna(abstracts['ABSTRACT_TEXT'])]

In [14]:
# fill null abstract and project values with empty strings '' for error avoidance
abstracts = abstracts[~pd.isna(abstracts['ABSTRACT_TEXT'])]
abstracts['ABSTRACT_TEXT'] = abstracts['ABSTRACT_TEXT'].astype(str).fillna('')
abstracts['ABSTRACT_TEXT'] = abstracts['ABSTRACT_TEXT'].replace("nan", "")
projects = projects[~pd.isna(projects['PROJECT_TITLE'])]

In [15]:
# merge data into single dataframe
proj_data = pd.merge(
    projects[['APPLICATION_ID', 'PROJECT_TITLE']], 
    abstracts[['APPLICATION_ID', 'ABSTRACT_TEXT']], 
    on='APPLICATION_ID', 
    how='left'
)

In [16]:
proj_data

,APPLICATION_ID,PROJECT_TITLE,ABSTRACT_TEXT
0,6465311,"Restin: mutants, receptor cloning and signalin...",Anti-angiogenic molecules have tremendous pote...
1,6626317,Mapping Novel Genes for Late-Onset Alzheimer's...,Late-onset Alzheimer's disease (LOAD) is the m...
2,6634305,NOVEL PHARMACOTHERAPY FOR TREATMENT OF COCAINE...,This application is in response to SPIRCAP RFA...
3,6803975,A Phase I Trial of CCL-21 Gene Modified DC in ...,DESCRIPTION (provided by applicant): Second...
4,6806737,Computer-based Intervention for Type 2 Diabete...,Diabetes is the seventh leading cause of death...
...,...,...,...
3046950,11078400,Disentangling specific and off-target signals ...,NaN
3046951,11079423,Malaria Operations Research to Improve Malaria...,NaN
3046952,11081606,"GH18-004, Surveillance and research for the in...",NaN
3046953,11111898,HIV and Cocaine Drive Bone-Marrow Blood (BMB) ...,NaN


In [21]:
temp_sample = proj_data.sample(n=100000, random_state=42)

In [22]:
# specify file path
output_file_path = "/Users/owensharpe/Desktop/Analyzing NIH Database/data collection/nih_reporter_website_data/annotations.jsonl"

# store annotation data in a json file
with open(output_file_path, "w") as outfile:
    
    # gather title and abstract annotation data
    for _, row in temp_sample.iterrows():   
        
        # first see if the project has an abstract
        if pd.isna(row['ABSTRACT_TEXT']) or not row['ABSTRACT_TEXT'].strip():
            abstract_annotations_dict = []
        else:
            abstract_annotations = gilda.annotate(row['ABSTRACT_TEXT'])
            abstract_annotations_dict = [annotation.to_json() for annotation in abstract_annotations]
        
        # now perform annotation on the project title
        title_annotations = gilda.annotate(row['PROJECT_TITLE'])
        title_annotations_dict = [annotation.to_json() for annotation in title_annotations]
        
        temp_project_data = {
            "application_id": row["APPLICATION_ID"],
            "abstract_annotations": abstract_annotations_dict,  # only abstracts
            "title_annotations": title_annotations_dict  # only titles
        }
    
        json.dump(temp_project_data, outfile)
        outfile.write("\n")

INFO: [2025-03-13 16:42:52] gilda.grounder - Running Gilda disambiguation for A2
INFO: [2025-03-13 16:42:52] gilda.grounder - Running Gilda disambiguation for A2
INFO: [2025-03-13 16:42:52] gilda.grounder - Running Gilda disambiguation for A2
INFO: [2025-03-13 16:42:52] gilda.grounder - Running Gilda disambiguation for A2
INFO: [2025-03-13 16:42:52] gilda.grounder - Running Gilda disambiguation for A2
INFO: [2025-03-13 16:42:52] gilda.grounder - Running Gilda disambiguation for CAR
INFO: [2025-03-13 16:42:52] gilda.grounder - Running Gilda disambiguation for CAR
INFO: [2025-03-13 16:42:52] gilda.grounder - Running Gilda disambiguation for CAR
INFO: [2025-03-13 16:42:52] gilda.grounder - Running Gilda disambiguation for CAR
INFO: [2025-03-13 16:42:52] gilda.grounder - Running Gilda disambiguation for CAR
INFO: [2025-03-13 16:42:52] gilda.grounder - Running Gilda disambiguation for SAP
INFO: [2025-03-13 16:42:52] gilda.grounder - Running Gilda disambiguation for SAP
INFO: [2025-03-13 16: